In [1]:
"""
===============================================================================
PINN for 1D ODE with Neumann Boundary Conditions
===============================================================================
This example demonstrates a Physics-Informed Neural Network (PINN) to solve a
simple one-dimensional ordinary differential equation (ODE) with Neumann boundary
conditions.

Problem Setup:
---------------
We consider the ODE:
    
    u''(x) + π² sin(π x) = 0,   x ∈ [0, 1]

whose exact solution is given by

    u(x) = sin(π x).

Accordingly, the derivative is

    u'(x) = π cos(π x).

Thus, the Neumann boundary conditions are:
    - At x = 0: u'(0) = π cos(0) = π,
    - At x = 1: u'(1) = π cos(π) = -π.

Boundary Condition Enforcement:
-------------------------------
There are two common approaches in PINNs:
1. Hard enforcement – building the trial solution to inherently satisfy the boundary
   conditions (often via output transformations).
2. Soft enforcement – adding penalty terms in the loss function that measure the
   discrepancy at the boundaries.

In this code we use soft enforcement. The Neumann conditions are enforced by:
   • Computing the network output u(x) at the boundary points.
   • Using automatic differentiation to obtain u'(x) at x = 0 and x = 1.
   • Penalizing the squared difference between these computed derivatives and
     the prescribed values (π at x = 0 and –π at x = 1).

The total loss function is the sum of:
   • The residual loss – enforcing that u(x) satisfies the differential equation
     at collocation points in the domain.
   • The boundary loss – enforcing that the derivative u'(x) at the boundaries
     matches the Neumann conditions.

This code uses PyTorch for building the network and performing automatic differentiation.
===============================================================================
"""

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define a simple fully-connected neural network for the PINN
class PINN1D(nn.Module):
    def __init__(self, layers):
        super(PINN1D, self).__init__()
        self.activation = nn.Tanh()
        layer_list = []
        for i in range(len(layers)-1):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
        self.layers = nn.ModuleList(layer_list)
    
    def forward(self, x):
        # x has shape (N,1)
        out = x
        for layer in self.layers[:-1]:
            out = self.activation(layer(out))
        out = self.layers[-1](out)
        return out

# Set seeds for reproducibility
torch.manual_seed(1234)
np.random.seed(1234)

# Generate collocation points in the domain [0, 1]
N_collocation = 1000
x_collocation = torch.linspace(0, 1, N_collocation).view(-1, 1)
# Enable gradient tracking for automatic differentiation
x_collocation.requires_grad = True

# Define boundary points for Neumann BC (here we use one point at each boundary)
x_bc = torch.tensor([[0.0], [1.0]], requires_grad=True)
# Prescribed derivative values computed from u'(x)=π cos(πx)
bc_values = torch.tensor([[np.pi], [-np.pi]], dtype=torch.float32)

# Instantiate the neural network: input layer (1 neuron), three hidden layers, output layer (1 neuron)
layers = [1, 20, 20, 20, 1]
model = PINN1D(layers)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)
n_iter = 5000

# Training loop
for epoch in range(n_iter):
    optimizer.zero_grad()
    
    # ---------------------------
    # PDE Residual Loss Computation
    # ---------------------------
    # Predict u(x) at collocation points
    u = model(x_collocation)
    # Compute first derivative u_x = du/dx
    u_x = torch.autograd.grad(u, x_collocation, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    # Compute second derivative u_xx = d²u/dx²
    u_xx = torch.autograd.grad(u_x, x_collocation, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    # Define the ODE residual: u''(x) + π² sin(πx) = 0
    f = torch.sin(np.pi * x_collocation)
    residual = u_xx + (np.pi**2) * f
    loss_res = torch.mean(residual**2)
    
    # ---------------------------
    # Boundary Loss for Neumann BC (Soft Enforcement)
    # ---------------------------
    # Evaluate network at the boundary points
    u_bc = model(x_bc)
    # Compute derivative u'(x) at the boundaries
    u_bc_x = torch.autograd.grad(u_bc, x_bc, grad_outputs=torch.ones_like(u_bc), create_graph=True)[0]
    # Mean squared error between computed derivative and prescribed BC values
    loss_bc = torch.mean((u_bc_x - bc_values)**2)
    
    # Total loss is a sum of PDE residual and boundary losses
    loss = loss_res + loss_bc
    loss.backward()
    optimizer.step()
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training finished")


Epoch 0, Loss: 57.96015167236328
Epoch 500, Loss: 0.0036900572013109922
Epoch 1000, Loss: 0.0003550474939402193
Epoch 1500, Loss: 0.000316944089718163
Epoch 2000, Loss: 0.000299551960779354
Epoch 2500, Loss: 0.00028349546482786536
Epoch 3000, Loss: 0.00026441249065101147
Epoch 3500, Loss: 0.00024170932010747492
Epoch 4000, Loss: 0.00021538091823458672
Epoch 4500, Loss: 0.0001862893404904753
Training finished


In [2]:
"""
===============================================================================
PINN for 2D PDE with Neumann Boundary Conditions
===============================================================================
This example demonstrates a Physics-Informed Neural Network (PINN) to solve a
two-dimensional partial differential equation (PDE) with Neumann boundary conditions.

Problem Setup:
---------------
We consider the PDE:

    u_xx(x,y) + u_yy(x,y) + 2π² sin(π x) cos(π y) = 0,   for (x,y) ∈ [0,1]×[0,1]

The exact solution is given by

    u(x,y) = sin(π x) cos(π y).

The corresponding derivatives are:
    u_x(x,y) = π cos(π x) cos(π y),
    u_y(x,y) = -π sin(π x) sin(π y).

Thus, the Neumann boundary conditions are:
    - On x = 0: u_x(0,y) = π cos(π y),
    - On x = 1: u_x(1,y) = -π cos(π y),
    - On y = 0 and y = 1: u_y(x,y) = 0  (since sin(0)=0 and sin(π)=0).

Boundary Condition Enforcement:
-------------------------------
As in the 1D case, we use soft enforcement. For each boundary:
   • We sample points on the boundary.
   • We compute the network output u(x,y) at those points.
   • Using automatic differentiation, we compute the appropriate normal derivative:
       - For vertical boundaries (x=0 and x=1), we compute u_x.
       - For horizontal boundaries (y=0 and y=1), we compute u_y.
   • We then form a penalty (mean squared error) between the computed derivative and
     the target Neumann value prescribed by the analytical solution.
     
The overall loss is the sum of:
   • The PDE residual loss (ensuring that the PDE is approximately satisfied in the
     interior of the domain).
   • The boundary loss (ensuring that the Neumann boundary conditions hold).

This code uses PyTorch for both the neural network and for computing derivatives.
===============================================================================
"""

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define a fully-connected neural network for the 2D PINN
class PINN2D(nn.Module):
    def __init__(self, layers):
        super(PINN2D, self).__init__()
        self.activation = nn.Tanh()
        layer_list = []
        for i in range(len(layers)-1):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
        self.layers = nn.ModuleList(layer_list)
    
    def forward(self, x):
        # x has shape (N,2) where columns correspond to (x,y)
        out = x
        for layer in self.layers[:-1]:
            out = self.activation(layer(out))
        out = self.layers[-1](out)
        return out

# Set seeds for reproducibility
torch.manual_seed(1234)
np.random.seed(1234)

# ---------------------------
# Generate Interior Collocation Points in [0,1]x[0,1]
# ---------------------------
N_collocation = 10000
n_side = int(np.sqrt(N_collocation))
x_coll = torch.linspace(0, 1, n_side)
y_coll = torch.linspace(0, 1, n_side)
X, Y = torch.meshgrid(x_coll, y_coll, indexing='ij')
# Reshape and concatenate to obtain interior points
X_interior = torch.cat((X.reshape(-1, 1), Y.reshape(-1, 1)), dim=1)
X_interior.requires_grad = True

# ---------------------------
# Generate Boundary Points for Neumann BC
# ---------------------------
N_bc = 200  # Number of points per boundary segment

# Left boundary (x = 0): sample y uniformly
y_bc_left = torch.rand((N_bc, 1))
x_bc_left = torch.zeros((N_bc, 1))
bc_left = torch.cat((x_bc_left, y_bc_left), dim=1)
bc_left.requires_grad = True

# Right boundary (x = 1): sample y uniformly
y_bc_right = torch.rand((N_bc, 1))
x_bc_right = torch.ones((N_bc, 1))
bc_right = torch.cat((x_bc_right, y_bc_right), dim=1)
bc_right.requires_grad = True

# Bottom boundary (y = 0): sample x uniformly
x_bc_bottom = torch.rand((N_bc, 1))
y_bc_bottom = torch.zeros((N_bc, 1))
bc_bottom = torch.cat((x_bc_bottom, y_bc_bottom), dim=1)
bc_bottom.requires_grad = True

# Top boundary (y = 1): sample x uniformly
x_bc_top = torch.rand((N_bc, 1))
y_bc_top = torch.ones((N_bc, 1))
bc_top = torch.cat((x_bc_top, y_bc_top), dim=1)
bc_top.requires_grad = True

# ---------------------------
# Define Target Boundary Values from the Analytical Solution:
# u(x,y) = sin(πx) cos(πy)
# Therefore:
#   For x-boundaries: u_x = π cos(πx) cos(πy)
#       At x=0: u_x(0,y) = π cos(0) cos(πy) = π cos(πy)
#       At x=1: u_x(1,y) = π cos(π) cos(πy) = -π cos(πy)
#   For y-boundaries: u_y = -π sin(πx) sin(πy)
#       At y=0 and y=1: u_y = 0 (since sin(0)=0 or sin(π)=0)
# ---------------------------
def bc_left_target(y):
    return np.pi * torch.cos(np.pi * y)

def bc_right_target(y):
    return -np.pi * torch.cos(np.pi * y)

def bc_bottom_target(x):
    return torch.zeros_like(x)

def bc_top_target(x):
    return torch.zeros_like(x)

# ---------------------------
# Instantiate the Neural Network
# ---------------------------
layers = [2, 20, 20, 20, 1]
model = PINN2D(layers)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
n_iter = 5000

# Training loop
for epoch in range(n_iter):
    optimizer.zero_grad()
    
    # ---------------------------
    # PDE Residual Loss Computation
    # ---------------------------
    u = model(X_interior)
    # Compute gradients with respect to x and y
    grads = torch.autograd.grad(u, X_interior, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = grads[:, 0:1]
    u_y = grads[:, 1:2]
    # Compute second derivatives: u_xx and u_yy
    u_xx = torch.autograd.grad(u_x, X_interior, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:, 0:1]
    u_yy = torch.autograd.grad(u_y, X_interior, grad_outputs=torch.ones_like(u_y), create_graph=True)[0][:, 1:2]
    # The PDE is: u_xx + u_yy + 2π² sin(πx) cos(πy) = 0
    f = torch.sin(np.pi * X_interior[:, 0:1]) * torch.cos(np.pi * X_interior[:, 1:2])
    residual = u_xx + u_yy + 2 * (np.pi**2) * f
    loss_pde = torch.mean(residual**2)
    
    # ---------------------------
    # Boundary Loss Computation (Soft Enforcement of Neumann BC)
    # ---------------------------
    # Left boundary (x=0): enforce u_x(0,y)= π cos(πy)
    u_left = model(bc_left)
    u_left_x = torch.autograd.grad(u_left, bc_left, grad_outputs=torch.ones_like(u_left), create_graph=True)[0][:, 0:1]
    target_left = bc_left_target(bc_left[:, 1:2])
    loss_left = torch.mean((u_left_x - target_left)**2)
    
    # Right boundary (x=1): enforce u_x(1,y)= -π cos(πy)
    u_right = model(bc_right)
    u_right_x = torch.autograd.grad(u_right, bc_right, grad_outputs=torch.ones_like(u_right), create_graph=True)[0][:, 0:1]
    target_right = bc_right_target(bc_right[:, 1:2])
    loss_right = torch.mean((u_right_x - target_right)**2)
    
    # Bottom boundary (y=0): enforce u_y(x,0)= 0
    u_bottom = model(bc_bottom)
    u_bottom_y = torch.autograd.grad(u_bottom, bc_bottom, grad_outputs=torch.ones_like(u_bottom), create_graph=True)[0][:, 1:2]
    target_bottom = bc_bottom_target(bc_bottom[:, 0:1])
    loss_bottom = torch.mean((u_bottom_y - target_bottom)**2)
    
    # Top boundary (y=1): enforce u_y(x,1)= 0
    u_top = model(bc_top)
    u_top_y = torch.autograd.grad(u_top, bc_top, grad_outputs=torch.ones_like(u_top), create_graph=True)[0][:, 1:2]
    target_top = bc_top_target(bc_top[:, 0:1])
    loss_top = torch.mean((u_top_y - target_top)**2)
    
    loss_bc = loss_left + loss_right + loss_bottom + loss_top
    
    # Total loss: sum of interior PDE residual loss and boundary loss
    loss = loss_pde + loss_bc
    loss.backward()
    optimizer.step()
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training finished")


Epoch 0, Loss: 106.51937103271484
Epoch 500, Loss: 0.12774918973445892
Epoch 1000, Loss: 0.02215679921209812
Epoch 1500, Loss: 0.010023819282650948
Epoch 2000, Loss: 0.00548941595479846
Epoch 2500, Loss: 0.003494677832350135
Epoch 3000, Loss: 0.0021805635187774897
Epoch 3500, Loss: 0.0015068539651110768
Epoch 4000, Loss: 0.0011680236784741282
Epoch 4500, Loss: 0.000977365649305284
Training finished


In [3]:
"""
===============================================================================
PINN for 1D ODE with Neumann Boundary Conditions - Hard Enforcement
===============================================================================
This example demonstrates a Physics-Informed Neural Network (PINN) to solve a
one-dimensional ordinary differential equation (ODE) with Neumann boundary
conditions using hard enforcement.

Problem Setup:
---------------
We consider the ODE:

    u''(x) + π² sin(π x) = 0,   x ∈ [0, 1]

with the exact solution

    u(x) = sin(π x).

Thus, the derivative is

    u'(x) = π cos(π x),

and the Neumann conditions are:
    - u'(0) = π,
    - u'(1) = -π.

Hard Enforcement Strategy:
--------------------------
Instead of penalizing the boundary derivative in the loss (soft enforcement),
we build the trial solution as:

    u_trial(x) = g(x) + h(x)*N(x),

where:
    • g(x) is a predetermined function that exactly satisfies the BCs:
          Let g(x) = π x - π x².
          Then, g'(0)=π and g'(1)=π-2π = -π.
    • h(x) is a function that vanishes in its derivative at the boundaries:
          Let h(x) = x²*(1-x)².
    • N(x) is the neural network output.
Thus, no matter what N(x) is, the derivative of u_trial(x) at the boundaries is:
    u_trial'(x) = g'(x) + [h'(x)*N(x) + h(x)*N'(x)],
and since h'(0)=h'(1)=0, we have
    u_trial'(0)=g'(0)=π,  u_trial'(1)=g'(1)=-π.

The loss function is then solely based on the PDE residual in the domain.
===============================================================================
"""

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the neural network for the correction term N(x)
class PINN1D(nn.Module):
    def __init__(self, layers):
        super(PINN1D, self).__init__()
        self.activation = nn.Tanh()
        layer_list = []
        for i in range(len(layers)-1):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
        self.net = nn.Sequential(*layer_list)
    
    def forward(self, x):
        # x has shape (N,1)
        out = x
        for layer in self.net[:-1]:
            out = self.activation(layer(out))
        out = self.net[-1](out)
        return out

# Functions for hard enforcement
def g(x):
    # g(x) = π x - π x²
    return np.pi * x - np.pi * x**2

def h(x):
    # h(x) = x²*(1-x)²
    return x**2 * (1 - x)**2

# Set seeds for reproducibility
torch.manual_seed(1234)
np.random.seed(1234)

# Generate collocation points in the domain [0, 1]
N_collocation = 1000
x_collocation = torch.linspace(0, 1, N_collocation).view(-1, 1)
x_collocation.requires_grad = True

# Instantiate the neural network: input layer (1 neuron), hidden layers, output layer (1 neuron)
layers = [1, 20, 20, 20, 1]
model = PINN1D(layers)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
n_iter = 5000

# Training loop for hard enforcement: only the PDE residual is penalized.
for epoch in range(n_iter):
    optimizer.zero_grad()
    
    # Compute the network output N(x)
    N_pred = model(x_collocation)
    # Compute the trial solution: u(x) = g(x) + h(x)*N(x)
    x_np = x_collocation.detach().cpu().numpy()
    g_val = torch.tensor(g(x_np), dtype=torch.float32)
    h_val = torch.tensor(h(x_np), dtype=torch.float32)
    # Ensure same shape as x_collocation
    g_val = g_val.view(-1, 1)
    h_val = h_val.view(-1, 1)
    u = g_val + h_val * N_pred
    
    # Compute derivatives: u_x and u_xx
    u_x = torch.autograd.grad(u, x_collocation, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x_collocation, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    
    # PDE residual: u''(x) + π² sin(πx) = 0
    f = torch.sin(np.pi * x_collocation)
    residual = u_xx + (np.pi**2) * f
    loss_res = torch.mean(residual**2)
    
    loss = loss_res
    loss.backward()
    optimizer.step()
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training finished - Hard Enforcement (1D)")


Epoch 0, Loss: 48.6347541809082
Epoch 500, Loss: 11.553899765014648
Epoch 1000, Loss: 5.396996021270752
Epoch 1500, Loss: 2.965437889099121
Epoch 2000, Loss: 1.7772620916366577
Epoch 2500, Loss: 1.1364892721176147
Epoch 3000, Loss: 0.7607347369194031
Epoch 3500, Loss: 0.5289739966392517
Epoch 4000, Loss: 0.37436825037002563
Epoch 4500, Loss: 0.27223652601242065
Training finished - Hard Enforcement (1D)


In [4]:
"""
===============================================================================
PINN for 2D PDE with Neumann Boundary Conditions - Hard Enforcement
===============================================================================
This example demonstrates a Physics-Informed Neural Network (PINN) to solve a
two-dimensional partial differential equation (PDE) with Neumann boundary
conditions using hard enforcement.

Problem Setup:
---------------
We consider the PDE:

    u_xx(x,y) + u_yy(x,y) + 2π² sin(πx) cos(πy) = 0,   for (x,y) ∈ [0,1]×[0,1]

with the exact solution

    u(x,y) = sin(πx) cos(πy).

The derivatives are:
    u_x(x,y) = π cos(πx) cos(πy),
    u_y(x,y) = -π sin(πx) sin(πy).

Thus, the Neumann boundary conditions are:
    - On x = 0: u_x(0,y) = π cos(πy),
    - On x = 1: u_x(1,y) = -π cos(πy),
    - On y = 0: u_y(x,0) = 0,
    - On y = 1: u_y(x,1) = 0.

Hard Enforcement Strategy:
--------------------------
We build the trial solution as:

    u_trial(x,y) = g(x,y) + h(x,y) * N(x,y),

where:
    • g(x,y) is chosen to satisfy the boundary derivative conditions:
          Let g(x,y)= π x cos(πy) - π x² cos(πy).
          Then, g_x(0,y)= π cos(πy) and g_x(1,y)= -π cos(πy);
          Also, g_y(x,0)=g_y(x,1)=0.
    • h(x,y) is chosen such that its normal derivatives vanish on all boundaries:
          A convenient choice is h(x,y)= x²(1-x)² * y²(1-y)².
    • N(x,y) is the neural network output.
Because h(x,y) is constructed so that its derivatives vanish along the boundaries,
the normal derivative of u_trial is u_trial_x=g_x and u_trial_y=g_y at the respective
boundaries—thus automatically satisfying the Neumann conditions.

The loss function is solely the interior PDE residual.
===============================================================================
"""

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the neural network for the correction term N(x,y)
class PINN2D(nn.Module):
    def __init__(self, layers):
        super(PINN2D, self).__init__()
        self.activation = nn.Tanh()
        layer_list = []
        for i in range(len(layers)-1):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
        self.net = nn.Sequential(*layer_list)
    
    def forward(self, x):
        # x has shape (N,2) where columns correspond to (x,y)
        out = x
        for layer in self.net[:-1]:
            out = self.activation(layer(out))
        out = self.net[-1](out)
        return out

# Hard enforcement functions for 2D
def g(xy):
    # g(x,y)= π x cos(πy) - π x² cos(πy)
    x = xy[:, 0:1]
    y = xy[:, 1:2]
    return np.pi * x * np.cos(np.pi * y) - np.pi * (x**2) * np.cos(np.pi * y)

def h(xy):
    # h(x,y)= x²*(1-x)² * y²*(1-y)²
    x = xy[:, 0:1]
    y = xy[:, 1:2]
    return (x**2 * (1 - x)**2) * (y**2 * (1 - y)**2)

# Set seeds for reproducibility
torch.manual_seed(1234)
np.random.seed(1234)

# ---------------------------
# Generate Interior Collocation Points in [0,1]x[0,1]
# ---------------------------
N_collocation = 10000
n_side = int(np.sqrt(N_collocation))
x_coll = torch.linspace(0, 1, n_side)
y_coll = torch.linspace(0, 1, n_side)
X, Y = torch.meshgrid(x_coll, y_coll, indexing='ij')
X_interior = torch.cat((X.reshape(-1, 1), Y.reshape(-1, 1)), dim=1)
X_interior.requires_grad = True

# Instantiate the neural network
layers = [2, 20, 20, 20, 1]
model = PINN2D(layers)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
n_iter = 5000

# Training loop for hard enforcement: only the PDE residual is penalized.
for epoch in range(n_iter):
    optimizer.zero_grad()
    
    # Compute network output N(x,y)
    N_pred = model(X_interior)
    
    # Compute g(x,y) and h(x,y) on interior points
    xy_np = X_interior.detach().cpu().numpy()
    g_val = torch.tensor(g(xy_np), dtype=torch.float32)
    h_val = torch.tensor(h(xy_np), dtype=torch.float32)
    g_val = g_val.view(-1, 1)
    h_val = h_val.view(-1, 1)
    
    # Compute trial solution: u(x,y)= g(x,y) + h(x,y)*N(x,y)
    u = g_val + h_val * N_pred
    
    # Compute gradients with respect to x and y
    grads = torch.autograd.grad(u, X_interior, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = grads[:, 0:1]
    u_y = grads[:, 1:2]
    # Second derivatives
    u_xx = torch.autograd.grad(u_x, X_interior, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:, 0:1]
    u_yy = torch.autograd.grad(u_y, X_interior, grad_outputs=torch.ones_like(u_y), create_graph=True)[0][:, 1:2]
    
    # PDE residual: u_xx + u_yy + 2π² sin(πx) cos(πy) = 0
    f = torch.sin(np.pi * X_interior[:, 0:1]) * torch.cos(np.pi * X_interior[:, 1:2])
    residual = u_xx + u_yy + 2 * (np.pi**2) * f
    loss_pde = torch.mean(residual**2)
    
    loss = loss_pde
    loss.backward()
    optimizer.step()
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training finished - Hard Enforcement (2D)")


Epoch 0, Loss: 97.39935302734375
Epoch 500, Loss: 92.94125366210938
Epoch 1000, Loss: 89.18375396728516
Epoch 1500, Loss: 67.68165588378906
Epoch 2000, Loss: 60.44888687133789
Epoch 2500, Loss: 55.34098815917969
Epoch 3000, Loss: 49.651947021484375
Epoch 3500, Loss: 41.922855377197266
Epoch 4000, Loss: 40.552467346191406
Epoch 4500, Loss: 31.22797203063965
Training finished - Hard Enforcement (2D)
